# Determine BATCH II Scan Errors

#### Updated: April 24, 2023 by Ashley Ferreira

#### Setup 
You will likely need to pip install tensorflow and keras_ocr as they do not come by default with anaconda, uncomment the cells below to do this if needed. Then run the cells to import the libraries adn set some of the default parameters.

In [ ]:
#! pip install tensorflow --user

In [ ]:
#! pip install keras_ocr --user

In [2]:
#! pip install line_profiler

In [1]:
# imports
import cv2
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import gc
from numba import jit, cuda, njit

# replace this with your own library path for --user pip installs
sys.path.append('C:/Users/aferreira/AppData/Roaming/Python/Python38/Scripts')
import keras_ocr

pipeline = keras_ocr.pipeline.Pipeline()

In [ ]:
%load_ext line_profiler
print(os.cpu_count())

In [3]:
# set paths
batchDir = 'L:/DATA/Alouette_I/BATCH_II_raw/'
save_dir = 'U:/Downloads/' 
outFile = save_dir + 'notebook20_outputs_v12.csv'

# set default saving settings
append2outFile = True
saveImages = False

Looking for C:\Users\aferreira\.keras-ocr\craft_mlt_25k.h5


Looking for C:\Users\aferreira\.keras-ocr\crnn_kurapan.h5


4


#### Inititalize functions
The main processing for this code uses two functions, read_all_rolls() which loops over all the batch 2 raw data ionogram images and saves the outputs from read_image() to a CSV file. This second function, read_image() reads in one image a time, whos path is passed it it by read_all_rolls(), and outputs the height and width along with the estimated digit count of the metadata. 

In [4]:
def read_image(image_path):
    '''
    '''
    try: 

        # read in image using keras_ocr
        image = keras_ocr.tools.read(image_path) 

        # extract height and width of image in pixels 
        height, width = image.shape[0], image.shape[1]

        # cut image to just include bottom 20% of pixels
        cropped_height = height-height//5
        cropped_image = [image[cropped_height:height,:]]

        # create predictions for location and value of characters
        # on the cropped image, will output (word, box) tuples
        prediction = pipeline.recognize([image])[0]

        # if no characters are found move on
        if prediction == [[]]:
            digit_count = 0

        # if characters are found look at the predictions
        else:

            # check how many are numbers since letters are often picked up from noise 
            # (sometimes something like a '0' maps to an 'o', but we only expect digits in metadata)

            # loop over predicted (word, box) tuples and count number of digit characters
            digit_count = 0
            says_isis = False
            for p in prediction:

                # select word and box part of the tuple
                value, box = p[0], p[1]

                # check for 'isis' of any capitalization in image
                # (can do the text detection process more efficiently)
                if 'isis' in value.lower():
                    says_isis = True
                    print('found potential ISIS text')
                

                # check that box is within the cropped height
                in_bounds = True
                for b in box:
                    if b[1] < cropped_height:
                        in_bounds = False
                        break
                        
                if in_bounds:
                    digit_count += len(value)

        print('digits count:', digit_count)

    except:
        print('ERR')
        digit_count, height, width, says_isis = 'ERR', 'ERR', 'ERR', 'ERR'

    return digit_count, height, width, says_isis

In [5]:
def read_all_rolls(outFile=outFile, append2outFile=True, batchDir=batchDir, max_images=None, save_each=100):
   '''
   This function loops over all images nested within batchDir
   and saves the outputs from read_image() to a CSV file.

   Parameters:

      outFile (str, optional): path to CSV file where results from this 
                               function can be stored 

      append2outFile (bool, optional): if True will append to data in outFile 
                                       (if any exists), otherwise overwrites

      batchDir (str, optional): path to directory of entire batch 
                                of ionogram scan images to analyze

      saveImages (bool, optional): True to save all images with irregular
                                    aspect ratios for visual inspection, 
                                    False otherwise

      max_images (int, optional): maximum number of images used to iterate over

      save_each (int, optional): save results to CSV after this number of images

   Returns:

      None

   '''
   # check if there is already data in the output file 
   if os.path.exists(outFile) and os.path.getsize(outFile)!=0:
      found = False
      header = False 

      df = pd.read_csv(outFile)
      last_entry = batchDir + df['roll'].iloc[-1] + '/' + df['subdir'].iloc[-1] + '/' + df['image'].iloc[-1]
      del df 
      # Returns the number of
      # objects it has collected
      # and deallocated
      collected = gc.collect()
      
      # Prints Garbage collector
      # as 0 object
      print("Garbage collector: collected",
               "%d objects." % collected)

   else: 
      found = True
      header = True
      last_entry = ''

   # initialize lists to save values to in loop
   rolls, subdirs, images = [], [], []
   heights, widths, digit_counts = [], [], []
   says_isis_lst = []

   images_saved = 0
   
   # loop over all rolls in the batch 2 raw data directory
   raw_contents = os.listdir(batchDir)
   for roll in raw_contents:

      # loop over all subdirectories within the roll
      roll_contents = os.listdir(batchDir + roll) 
      for subdir in roll_contents:
         
         # loop over all images in the subdirectory
         subdir_contents = os.listdir(batchDir + roll + '/' + subdir) 
         for image in subdir_contents:

            # save full path of image
            image_path = batchDir + roll + '/' + subdir + '/' + image

            # skip over image if already analyzed in CSV
            if found == False and last_entry == image_path:
               found = True

            if found == True:
               images_saved += 1

               if max_images != None and images_saved > max_images:
                  sys.exit()

               # save id of image
               rolls.append(roll)
               subdirs.append(subdir)
               images.append(image)

               # send to read_image to get aspect ratio, digit count, and isis text
               num_of_digits, h, w, says_isis = read_image(image_path)

               # save values
               digit_counts.append(num_of_digits)
               heights.append(h)
               widths.append(w)
               says_isis_lst.append(says_isis)              

               # save to csv after a set number of images (perhaps best to make propto max images)
               if images_saved % save_each == 0:

                  # initialize dataframe and save results to csv
                  # (redoing this each interation to not loose information)
                  df_mapping_results = pd.DataFrame()

                  df_mapping_results['roll'] = rolls
                  df_mapping_results['subdir'] = subdirs
                  df_mapping_results['image'] = images
                  df_mapping_results['digit_count'] = digit_counts
                  df_mapping_results['height'] = heights
                  df_mapping_results['width'] = widths
                  df_mapping_results['says_isis'] = says_isis_lst

                  # mode = 'a' means it will append to existing data within the file
                  if append2outFile == True:
                     mode = 'a' 

                     # wipe lists now that they have been saved
                     rolls, subdirs, images = [], [], []
                     heights, widths, digit_counts = [], [], []
                     says_isis_lst = []
                     
                  else: 
                     # this overwrites existing file
                     mode = 'w'
                     header = True

                  df_mapping_results.to_csv(outFile, mode=mode, index=False, header=header)
                  del df_mapping_results

                  collected = gc.collect()
                  print("Garbage collector: collected",
                           "%d objects." % collected)

#### Running the functions
Below, I run the read_all_rolls() function for the batch 2 raw data directory and it saves the results as it processes.

In [10]:
# profile the read_all_rolls() function
# run it for 10 only to get results
%lprun -f read_all_rolls read_all_rolls(max_images=10, save_each=5)

1/1 [==============================] - 10s 10s/step
digits count: 18
1/1 [==============================] - 1s 1s/step
digits count: 0
1/1 [==============================] - 3s 3s/step
digits count: 17
1/1 [==============================] - 1s 1s/step
digits count: 18
1/1 [==============================] - 2s 2s/step
digits count: 22
Garbage collector: collected 6448 objects.
1/1 [==============================] - 1s 1s/step
digits count: 21
1/1 [==============================] - 2s 2s/step
digits count: 20
1/1 [==============================] - 2s 2s/step
digits count: 20
1/1 [==============================] - 2s 2s/step
digits count: 26
1/1 [==============================] - 1s 1s/step
digits count: 10
Garbage collector: collected 1449 objects.
*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-07 s

Total time: 496.886 s
File: <ipython-input-9-414e5736e081>
Function: read_all_rolls at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def read_all_rolls(outFile=outFile, append2outFile=True, batchDir=batchDir, max_images=None, save_each=100):
     2                                              '''
     3                                              This function loops over all images nested within batchDir
     4                                              and saves the outputs from read_image() to a CSV file.
     5                                           
     6                                              Parameters:
     7                                           
     8                                                 outFile (str, optional): path to CSV file where results from this 
     9                                                                          function can be stored 

In [13]:
# (just one iteration here)
image_path = batchDir + 'R014207709' + '/' + '145' + '/' + '1.png'
%lprun -f read_image read_image(image_path)

1/1 [==============================] - 2s 2s/step
digits count: 18


Timer unit: 1e-07 s

Total time: 45.2449 s
File: <ipython-input-6-be4c62cda9ae>
Function: read_image at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def read_image(image_path):
     4                                               '''
     5                                               '''
     6         1         20.0     20.0      0.0      try: 
     7                                           
     8                                                   # read in image using keras_ocr
     9         1    7477189.0 7477189.0      1.7          image = keras_ocr.tools.read(image_path) 
    10                                           
    11                                                   # extract height and width of image in pixels 
    12         1         84.0     84.0      0.0          height, width = image.shape[0], image.shape[1]
    13                                           
    14                      

In [12]:
# (just one iteration here)
image_path = batchDir + 'R014207709' + '/' + '145' + '/' + '10.png'
%lprun -f read_image read_image(image_path)

1/1 [==============================] - 8s 8s/step
digits count: 0


Timer unit: 1e-07 s

Total time: 15.3668 s
File: <ipython-input-11-e374146448d1>
Function: read_image at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def read_image(image_path):
     2                                               '''
     3                                               '''
     4         1          7.0      7.0      0.0      try: 
     5                                           
     6                                                   # read in image using keras_ocr
     7         1    3859733.0 3859733.0      2.5          image = keras_ocr.tools.read(image_path) 
     8                                           
     9                                                   # extract height and width of image in pixels 
    10         1         43.0     43.0      0.0          height, width = image.shape[0], image.shape[1]
    11                                           
    12                     

#### Multiprocessing & Multithreading Implimentations
Testing to see if either of these are a worthwhile approach. Most experimentation done by edditing this on VDI so don't pay much attention to the code in this section.

In [15]:
from multiprocessing import Process
import threading

In [20]:
def read_all_rolls_alt(outFile=outFile, append2outFile=True, batchDir=batchDir, max_images=None, save_each=100):
   '''
   '''
   def roll_process(roll):
         # loop over all subdirectories within the roll
         roll_contents = os.listdir(batchDir + roll) 
         for subdir in roll_contents:
            
            # loop over all images in the subdirectory
            subdir_contents = os.listdir(batchDir + roll + '/' + subdir) 
            for image in subdir_contents:

               # save full path of image
               image_path = batchDir + roll + '/' + subdir + '/' + image

               # skip over image if already analyzed in CSV
               if found == False and last_entry == image_path:
                  found = True

               if found == True:
                  images_saved += 1

                  if max_images != None and images_saved > max_images:
                     sys.exit()

                  # save id of image
                  rolls.append(roll)
                  subdirs.append(subdir)
                  images.append(image)

                  # send to read_image to get aspect ratio, digit count, and isis text
                  num_of_digits, h, w, says_isis = read_image(image_path)

                  # save values
                  digit_counts.append(num_of_digits)
                  heights.append(h)
                  widths.append(w)
                  says_isis_lst.append(says_isis)              

                  # save to csv after a set number of images (perhaps best to make propto max images)
                  if images_saved % save_each == 0:

                     # initialize dataframe and save results to csv
                     # (redoing this each interation to not loose information)
                     df_mapping_results = pd.DataFrame()

                     df_mapping_results['roll'] = rolls
                     df_mapping_results['subdir'] = subdirs
                     df_mapping_results['image'] = images
                     df_mapping_results['digit_count'] = digit_counts
                     df_mapping_results['height'] = heights
                     df_mapping_results['width'] = widths
                     df_mapping_results['says_isis'] = says_isis_lst

                     # mode = 'a' means it will append to existing data within the file
                     if append2outFile == True:
                        mode = 'a' 

                        # wipe lists now that they have been saved
                        rolls, subdirs, images = [], [], []
                        heights, widths, digit_counts = [], [], []
                        says_isis_lst = []
                        
                     else: 
                        # this overwrites existing file
                        mode = 'w'
                        header = True

                     df_mapping_results.to_csv(outFile, mode=mode, index=False, header=header)
                     del df_mapping_results

                     collected = gc.collect()
                     print("Garbage collector: collected",
                              "%d objects." % collected)

   # check if there is already data in the output file 
   if os.path.exists(outFile) and os.path.getsize(outFile)!=0:
      found = False
      header = False 

      df = pd.read_csv(outFile)
      last_entry = batchDir + df['roll'].iloc[-1] + '/' + df['subdir'].iloc[-1] + '/' + df['image'].iloc[-1]
      del df 
      # Returns the number of
      # objects it has collected
      # and deallocated
      collected = gc.collect()
      
      # Prints Garbage collector
      # as 0 object
      print("Garbage collector: collected",
               "%d objects." % collected)

   else: 
      found = True
      header = True
      last_entry = ''

   # initialize lists to save values to in loop
   rolls, subdirs, images = [], [], []
   heights, widths, digit_counts = [], [], []
   says_isis_lst = []

   images_saved = 0
   
   # loop over all rolls in the batch 2 raw data directory
   raw_contents = os.listdir(batchDir)
   for input_roll in raw_contents:

      process = Process(target=roll_process, args=(input_roll, ))

      # Running the new process
      process.start()

      # Check process is running
      process.is_alive()

In [23]:
def read_all_rolls_alt2(roll, outFile=outFile, append2outFile=True, batchDir=batchDir, max_images=None, save_each=100):
   '''
   '''

   # initialize lists to save values to in loop
   rolls, subdirs, images = [], [], []
   heights, widths, digit_counts = [], [], []
   says_isis_lst = []

   images_saved = 0
   found = True
   header = True
   last_entry = ''

   # loop over all subdirectories within the roll
   roll_contents = os.listdir(batchDir + roll) 
   for subdir in roll_contents:
      
      # loop over all images in the subdirectory
      subdir_contents = os.listdir(batchDir + roll + '/' + subdir) 
      for image in subdir_contents:

         # save full path of image
         image_path = batchDir + roll + '/' + subdir + '/' + image

         # skip over image if already analyzed in CSV
         if found == False and last_entry == image_path:
            found = True

         if found == True:
            images_saved += 1

            if max_images != None and images_saved > max_images:
               sys.exit()

            # save id of image
            rolls.append(roll)
            subdirs.append(subdir)
            images.append(image)

            # send to read_image to get aspect ratio, digit count, and isis text
            num_of_digits, h, w, says_isis = read_image(image_path)

            # save values
            digit_counts.append(num_of_digits)
            heights.append(h)
            widths.append(w)
            says_isis_lst.append(says_isis)              

            # save to csv after a set number of images (perhaps best to make propto max images)
            if images_saved % save_each == 0:

               # initialize dataframe and save results to csv
               # (redoing this each interation to not loose information)
               df_mapping_results = pd.DataFrame()

               df_mapping_results['roll'] = rolls
               df_mapping_results['subdir'] = subdirs
               df_mapping_results['image'] = images
               df_mapping_results['digit_count'] = digit_counts
               df_mapping_results['height'] = heights
               df_mapping_results['width'] = widths
               df_mapping_results['says_isis'] = says_isis_lst

               # mode = 'a' means it will append to existing data within the file
               if append2outFile == True:
                  mode = 'a' 

                  # wipe lists now that they have been saved
                  rolls, subdirs, images = [], [], []
                  heights, widths, digit_counts = [], [], []
                  says_isis_lst = []
                  
               else: 
                  # this overwrites existing file
                  mode = 'w'
                  header = True

               df_mapping_results.to_csv(outFile, mode=mode, index=False, header=header)
               del df_mapping_results

               collected = gc.collect()
               print("Garbage collector: collected",
                        "%d objects." % collected)

In [12]:
def print_roll(roll):
    print(roll)

In [13]:
test_rolls = [
    'R014207709', 
    'R014207711'
]

# create processes
processes = [Process(target=read_all_rolls_alt2, args=[roll]) 
            for roll in test_rolls]

# start the processes
for process in processes:
    process.start()

# wait for completion
for process in processes:
    process.join()

In [14]:
# Check process is running
process.is_alive()

False

In [ ]:
t1 = threading.Thread(target=read_all_rolls_alt2, args=(test_rolls[0],))
t2 = threading.Thread(target=read_all_rolls_alt2, args=(test_rolls[1],))

# starting thread 1
t1.start()
# starting thread 2
t2.start()

# wait until thread 1 is completely executed
t1.join()
# wait until thread 2 is completely executed
t2.join()

# both threads completely executed
print("Done!")